In [84]:
pip install lxml

In [85]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import lxml
import re

## Meeting Plan for Nicolas Ouellet and Benedicte Knudson:
We colaborated over Zoom and in person **two** times and used a shared Github Repository that can be found [here](https://github.com/NickOuellet/Fly-Me-To-The-Moon). <br />
We met: <br />
10/11: 5-7pm <br />
10/20: 10-12pm <br />

### Step 1

In [86]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html",timeout=10,headers=headers)
r.status_code

200

In [87]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")

In [88]:
table = soup.findAll("table") #find the table in the HTML

In [89]:
solar_flare_df = pd.read_html(str(table))

In [90]:
solfla_df = solar_flare_df[0]#works when we just select the first table

In [91]:
solfla_df.columns = ["rank", "x_classification", "date", "region", "start_time", "maximum_time", "end_time", "movie"] #rename columns to be more descriptive

In [92]:
solfla_df.head()

,rank,x_classification,date,region,start_time,maximum_time,end_time,movie
0,1,X28+,2003/11/04,486,19:29,19:53,20:06,MovieView archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,MovieView archive
2,3,X17.2+,2003/10/28,486,09:51,11:10,11:24,MovieView archive
3,4,X17+,2005/09/07,808,17:17,17:40,18:03,MovieView archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,MovieView archive


### Step 2

In [93]:
solfla_df = solfla_df[["rank", "x_classification", "date", "start_time", "maximum_time", "end_time","region"]]
solfla_df.head()

,rank,x_classification,date,start_time,maximum_time,end_time,region
0,1,X28+,2003/11/04,19:29,19:53,20:06,486
1,2,X20+,2001/04/02,21:32,21:51,22:03,9393
2,3,X17.2+,2003/10/28,09:51,11:10,11:24,486
3,4,X17+,2005/09/07,17:17,17:40,18:03,808
4,5,X14.4,2001/04/15,13:19,13:50,13:55,9415


In [94]:
import datetime

In [95]:
for index, row in solfla_df.iterrows():
    solfla_df["start_time"] = pd.to_datetime(solfla_df["start_time"])
    solfla_df["maximum_time"] = pd.to_datetime(solfla_df["maximum_time"])
    solfla_df["end_time"] = pd.to_datetime(solfla_df["end_time"])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [96]:
solfla_df.drop(columns="date",inplace=True) #drop date table

solfla_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,rank,x_classification,start_time,maximum_time,end_time,region
0,1,X28+,2021-10-22 19:29:00,2021-10-22 19:53:00,2021-10-22 20:06:00,486
1,2,X20+,2021-10-22 21:32:00,2021-10-22 21:51:00,2021-10-22 22:03:00,9393
2,3,X17.2+,2021-10-22 09:51:00,2021-10-22 11:10:00,2021-10-22 11:24:00,486
3,4,X17+,2021-10-22 17:17:00,2021-10-22 17:40:00,2021-10-22 18:03:00,808
4,5,X14.4,2021-10-22 13:19:00,2021-10-22 13:50:00,2021-10-22 13:55:00,9415


### Step 3

In [97]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html",timeout=10,headers=headers)
r.status_code

200

In [99]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")
table = soup.get_text()
split_table = table.splitlines()
split_table = split_table[15:]
split_table = split_table[0:len(split_table)-2] #clean up table by removing top and bottom portion


In [108]:
nasa_df = pd.DataFrame(
    columns = ["start_date","start_time","end_date","end_time","start_freq","end_freq","location","region","xray_importance","cme_date","cme_time","central_pos_angle","cme_width","cme_speed","PHTX" ])
for i,v in enumerate(split_table):
  row_vals = re.findall(r"\S+",row)
  nasa_df.loc[i] = row_vals

In [111]:
nasa_df.drop(columns=["PHTX"],inplace=True)

In [121]:
nasa_df.loc[0:5]

,start_date,start_time,end_date,end_time,start_freq,end_freq,location,region,xray_importance,cme_date,cme_time,central_pos_angle,cme_width,cme_speed
0,2020/12/07,16:18,12/08,02:00,14000,160,S25W08,12790,C7.4,12/07,16:24,Halo,360,1407
1,2020/12/07,16:18,12/08,02:00,14000,160,S25W08,12790,C7.4,12/07,16:24,Halo,360,1407
2,2020/12/07,16:18,12/08,02:00,14000,160,S25W08,12790,C7.4,12/07,16:24,Halo,360,1407
3,2020/12/07,16:18,12/08,02:00,14000,160,S25W08,12790,C7.4,12/07,16:24,Halo,360,1407
4,2020/12/07,16:18,12/08,02:00,14000,160,S25W08,12790,C7.4,12/07,16:24,Halo,360,1407


## Step 4 - Tidy Nasa Data

In [122]:
nasa_df.replace("----", np.nan,inplace=True),axis=0)

NameError: ignored